##掛載 Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##加入專案路徑

In [2]:
import os
import sys
from flask import Flask, request, abort

# 🌟 0. 環境設定與診斷 (強制在匯入前執行) 🌟
# 請確保 Cell 1: drive.mount('/content/drive') 已經成功運行

PROJECT_PATH = '/content/drive/MyDrive/projects/LINE BOT'

# 診斷：檢查檔案是否真的存在於 Python 視角下的路徑
FILE_CHECK_NAME = 'v1_sqlite_dataseat.py'
FILE_CHECK_PATH = os.path.join(PROJECT_PATH, FILE_CHECK_NAME)

# --- 診斷邏輯 ---
if os.path.exists(FILE_CHECK_PATH):
    print(f"✅ 診斷成功：文件 '{FILE_CHECK_NAME}' 存在於 {PROJECT_PATH}。")
    if PROJECT_PATH not in sys.path:
        sys.path.append(PROJECT_PATH)
        print(f"✅ 已將路徑加入系統搜尋清單。")
else:
    # 如果文件不存在，立即停止並報告錯誤
    print(f"❌ 診斷失敗：文件 '{FILE_CHECK_NAME}' 不存在於 {PROJECT_PATH}。")
    print("請仔細檢查 Google Drive 上的資料夾名稱是否有隱藏的空格或符號。")
    raise FileNotFoundError("路徑或檔案名稱錯誤，無法繼續。")
# --- 診斷結束 ---

# 1. 導入您的資料庫函式 (現在路徑已設定，匯入應該成功)
from v1_sqlite_dataseat import init_db, insert_log, close_db, get_user_log_count

# ... (LINE Bot SDK 導入和 Flask 程式碼接在後面)
# ...

# (請確保 Flask/LINE Bot 程式碼是接在後面，而不是只有這段)

✅ 診斷成功：文件 'v1_sqlite_dataseat.py' 存在於 /content/drive/MyDrive/projects/LINE BOT。
✅ 已將路徑加入系統搜尋清單。


##1) 安裝套件（指定版本範圍＋用途註解）

In [3]:
# 安裝 LINE Bot SDK v3（官方最新主線）
!pip install "line-bot-sdk>=3.0.0,<4.0.0"

# 安裝 Flask（Web 框架，提供 webhook 端點）
!pip install "Flask>=3.0.0,<4.0.0"

# 安裝 pyngrok（把本機/Colab 5000 port 映射成公開 HTTPS）
!pip install "pyngrok>=7.0.0,<8.0.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.7/816.7 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 14.8 MB/s eta 0:00:00


##2) 開 ngrok（先執行這一格拿到公開網址）

In [7]:
from pyngrok import ngrok

# 換成你自己的 ngrok authtoken
NGROK_AUTHTOKEN = "2nh1S1yuCNhyHz7E9kibC5G9jfJ_5poDL8LVgJgfAm2a1ZJRc"   # 例：2pEWt4...（不要公開在 GitHub）
PORT = 5000

# 設定金鑰
ngrok.set_auth_token(NGROK_AUTHTOKEN)

# 關閉舊隧道（避免重複）
for t in ngrok.get_tunnels():
    ngrok.disconnect(t.public_url)

# 建立新隧道，取得公開 HTTPS 網址
public_url = ngrok.connect(PORT, "http").public_url

print("👉 外網網址：", public_url)
print("Webhook URL：", public_url + "/callback")

👉 外網網址： https://af37cb52ff6f.ngrok-free.app
Webhook URL： https://af37cb52ff6f.ngrok-free.app/callback


##3) LINE Bot Echo 程式（v3 SDK 正式寫法）

In [8]:
#從Line bot sdk pypl網站找到程式碼貼上
#https://pypi.org/project/line-bot-sdk/
import os
from flask import Flask, request, abort

# 🌟 導入您的資料庫函式 (使用您確認的檔名)
from v1_sqlite_dataseat import init_db, insert_log, close_db, get_user_log_count

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

# 憑證設定：請替換成您的 LINE Bot 憑證
CHANNEL_ACCESS_TOKEN = os.environ.get('LINE_ACCESS_TOKEN', '+Ri09o6iK7SXuNUaZEjWtlf9mZLp1xAt8NPXDVCN0shGvq1wvris4Oo29FZWdNqm7BW5a559L0klMfDltAo4qvsr1GI7OXMW2lW/mcZZYd2pwDpjrPQdxmKQ5W3vpYRC1VSq4rM84ksy+BybGREwygdB04t89/1O/w1cDnyilFU=')
CHANNEL_SECRET = os.environ.get('LINE_CHANNEL_SECRET', '4bc05bd9e903340338507195040bd2ff')

configuration = Configuration(access_token=CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

# 🌟 數據庫優化：註冊資料庫關閉函式 🌟
# 確保每個請求結束後都會關閉連線，釋放資源
app.teardown_appcontext(close_db)


@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    """
    處理使用者傳送的文字訊息：
    1. 儲存訊息到資料庫 (V1 核心)。
    2. 回覆個人化的日記總數。
    """
    user_id = event.source.user_id
    user_diary = event.message.text

    # 1. 🌟 儲存日記：將訊息寫入 Google Drive 上的 SQLite 資料庫
    insert_log(user_id=user_id, raw_text=user_diary)

    # 2. 查詢日記總數
    total_logs = get_user_log_count(user_id)

    # 3. 回覆使用者
    reply_text = (
        f"📝 您的日記已成功收錄！\n"
        f"這是您寫下的第 {total_logs} 篇紀錄。\n"
        f"我們會溫柔地為您保管這份珍貴的回憶。"
    )

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=reply_text)]
            )
        )

if __name__ == "__main__":
    # 🌟 應用程式啟動前，先初始化資料庫 🌟
    init_db()

    # 運行 Flask 服務
    app.run(port=5000)

✅ 資料庫 /content/drive/MyDrive/projects/LineBot_Data/emotion_diary.db 初始化完成，資料將永久儲存在 Google Drive 上。
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Oct/2025 02:49:20] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2025 02:49:32] "POST /callback HTTP/1.1" 200 -
